# Preminary operations

Let us start by importing the relevant packages

In [ ]:
import gcp
import gcp.bigquery as bq
import gcp.storage as storage
import pandas as pd
import matplotlib.pylab as plt
import matplotlib.colors as colors
import numpy as np
import seaborn as sns

## Let us explore the dataset

In [ ]:
%%sql 
SELECT * FROM [google.com:ddm-connectors-demo:aw_dataset.__TABLES__];

## Lest us explore the table KEYWORDS_PERFORMANCE_ANALYTICS

In [ ]:
%bigquery schema --table google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_PERFORMANCE_ANALYTICS

## Let us see how the data look like

In [ ]:
%bigquery sample --table google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_PERFORMANCE_ANALYTICS --count 5

# Let us start checking the monthly performance on Volagratis in Italy

In [ ]:
%sql --module monthly_keywords_performance
SELECT
  AccountDescriptiveName,
  CampaignName,
  AdGroupName,
  Criteria,
  KeywordMatchType,
  COUNT(Month) as MonthsWithImpressions,
  AVG(Impressions) as Impressions,
  AVG(ImpressionAssistedConversions) as ImpressionAssistedConversions,
  AVG(Clicks) as Clicks,
  AVG(ClickAssistedConversions) as ClickAssistedConversions,  
  AVG(CTR) as CTR,
  AVG(CostEUR/Clicks) as CPC,
  AVG(QualityScore) as QualityScore, 
  AVG(CostEUR) as CostEUR,
  AVG(AllConversions) as AllConversions,
  AVG(Conversions) as Conversions,    
  AVG(CostPerAllConversion) as CostPerAllConversion,
  AVG(CostPerConversion) as CostPerConversion,  
  #Analytics Metrics
  AVG(BounceRate) as BounceRate,
  AVG(AveragePosition) as AveragePosition,
  AVG(AveragePageviews) as AveragePageviews,
  AVG(AverageTimeOnSite) as AverageTimeOnSite,
  AVG(PercentNewVisitors) as PercentNewVisitors,
FROM
(
SELECT
  Month,
  AccountDescriptiveName,
  CampaignName,
  AdGroupName,
  Criteria,
  KeywordMatchType,
  SUM(Impressions) as Impressions,
  SUM(ImpressionAssistedConversions) as ImpressionAssistedConversions,
  SUM(Clicks) as Clicks,
  SUM(Clicks)/SUM(Impressions) as CTR,
  SUM(IF(ClickAssistedConversionsOverLastClickConversions = 0, 0,
  CASE
    WHEN Conversions > 0 OR AllConversions > 0 THEN IF(AllConversions>Conversions,AllConversions,Conversions)*ClickAssistedConversionsOverLastClickConversions
    WHEN ImpressionAssistedConversions > 0 THEN ImpressionAssistedConversions*ClickAssistedConversionsOverLastClickConversions
    ELSE 1.0
  END
  )) as ClickAssistedConversions,
  
  AVG(QualityScore) as QualityScore,
  
  SUM(IF(AccountCurrencyCode = 'GBP', Cost*1.37, Cost*1.0))/1000000 as CostEUR,
  SUM(AllConversions) as AllConversions,
  SUM(Conversions) as Conversions,  
  
  SUM(IF(AccountCurrencyCode = 'GBP', Cost*1.37, Cost*1.0))/1000000/SUM(AllConversions) as CostPerAllConversion,
  SUM(IF(AccountCurrencyCode = 'GBP', Cost*1.37, Cost*1.0))/1000000/SUM(Conversions) as CostPerConversion,
  
  #Analytics Metrics
  SUM(FLOAT(RTRIM(BounceRate,'%'))/100.0 * Clicks)/SUM(Clicks) as BounceRate,
  SUM(AveragePosition * Impressions)/SUM(Impressions) as AveragePosition,
  SUM(AveragePageviews * Clicks)/SUM(Clicks) as AveragePageviews,
  SUM(AverageTimeOnSite * Clicks)/SUM(Clicks) as AverageTimeOnSite,
  SUM(FLOAT(RTRIM(PercentNewVisitors,'%'))/100.0 * Clicks)/SUM(Clicks) as PercentNewVisitors,

FROM
  [google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_PERFORMANCE_ANALYTICS]
WHERE TRUE
AND NOT IsNegative
AND AdNetworkType2 = 'Google search'
AND Impressions > 0 
AND Month IN ('2015-08-01','2015-09-01','2015-10-01','2015-11-01')
AND AccountDescriptiveName IN ('Account Primario Volagratis','Volagratis Tratte Italia','Volagratis')
GROUP BY 1,2,3,4,5,6)
GROUP BY 1,2,3,4,5

In [ ]:
df = bq.Query(monthly_keywords_performance).to_dataframe()

## Does it make sense to differentiate among Brand and Generic?

Let us identify the Brand Keywords by assuming that 
- they have at least 10 impressions a month
- their CTR is higher the "normal"

In [ ]:
df[df['Impressions']>10].hist('CTR',bins=np.arange(101)/100.0,
                              normed=True,cumulative=1,histtype='step')

In [ ]:
df[(df['AdGroupName'] == 'Brand')&(df['Impressions']>10)].hist('CTR',bins=np.arange(101)/100.0,
                                                              cumulative=1,normed=1,
                                                               histtype='step')

In [ ]:
df[(df['AdGroupName'] == 'Brand')&
   (df['CTR']<0.4)&
   (df['Impressions']>10)].sort_values(by='CostEUR',ascending=False)[['AccountDescriptiveName',
                                                                                        'CampaignName',
                                                                                        'AdGroupName',
                                                                                        'Criteria',
                                                                                        'KeywordMatchType',
                                                                                        'CTR',
                                                                                        'Impressions',
                                                                                        'CostEUR']]

In [ ]:
df[(df['Impressions']>10) & (df['CTR']>0.4)].sort_values(by='CostEUR',ascending=False)[['AccountDescriptiveName',
                                                                                        'CampaignName',
                                                                                        'AdGroupName',
                                                                                        'Criteria',
                                                                                        'KeywordMatchType',
                                                                                        'CTR',
                                                                                        'Impressions',
                                                                                        'CostEUR']]

In [ ]:
df['KeywordType'] = df.apply(lambda x: 'Gold' if 
                             (x['Impressions']>10 and x['CTR']>0.4) 
                             else 'Standard', axis=1)


Let us start assessing the performance of the converting keywords

In [ ]:
df.groupby(by=(df['AllConversions'] > 0))[['AllConversions','ClickAssistedConversions',
                                           'ImpressionAssistedConversions','CostEUR',
                                           'Impressions','Clicks',
                                           'Criteria']].aggregate({'AllConversions':sum,
                                                                   'ClickAssistedConversions':sum,
                                                                   'ImpressionAssistedConversions':sum,
                                                                   'CostEUR':sum,
                                                                   'Impressions':sum,
                                                                   'Clicks':sum,
                                                                   'Criteria':'count'})

In [ ]:
df[df['AllConversions'] > 0].hist('Conversions',bins=100)

In [ ]:
axes = df[df['AllConversions'] > 0].hist('AllConversions',bins=np.logspace(0,4),log=True,by='KeywordType')
axes[0].set_xscale('log')
axes[1].set_xscale('log')

In [ ]:
axes = df[df['AllConversions'] > 0].hist('AllConversions',bins=np.logspace(0,4),by='KeywordType',
              cumulative=1,normed=True,histtype='step')
axes[0].set_xscale('log')
axes[1].set_xscale('log')

In [ ]:
df[df['AllConversions'] > 0].hist('AllConversions',bins=np.logspace(0,4),
              cumulative=1,histtype='step')
plt.gca().set_xscale('log')

In [ ]:
def setGroup(x):
  if x['AllConversions'] > 0:
    return 'Converter'
  else:
    return 'Other'
  
def setSubGroup(x):
  if x['AllConversions'] > 99:
    return 'Converter - High'
  elif x['AllConversions'] > 9:
    return 'Converter - Mid'
  elif x['AllConversions'] > 2:
    return 'Converter - Low'
  elif x['AllConversions'] > 0:
    return 'Converter - Occasional'
  else:
    return 'Other'
  
df['Group'] = df.apply(setGroup,axis=1)
df['SubGroup'] = df.apply(setSubGroup,axis=1)

## Let as check how the attributes of the Converters looks like 

In [ ]:
g = sns.pairplot(data=df[df['Group'] == 'Converter'][[u'SubGroup','CTR',
                                                  'CostPerAllConversion',
                            u'QualityScore',u'AveragePosition',                          
                            u'BounceRate', u'AveragePageviews',
                            u'AverageTimeOnSite',u'PercentNewVisitors']],hue=u'SubGroup')

In [ ]:
g = sns.pairplot(data=df[df['Group'] == 'Converter'][[u'SubGroup',                         
                            u'BounceRate', u'AveragePageviews',
                            u'AverageTimeOnSite',u'PercentNewVisitors']],hue=u'SubGroup')

In [ ]:
g = sns.pairplot(data=df[df['Group'] == 'Converter'][[u'SubGroup','CTR',
                                                  'CostPerAllConversion',
                            u'QualityScore',u'AveragePosition',                          
                            u'BounceRate',u'PercentNewVisitors']],hue=u'SubGroup')

## Can we move away from LastClick?

In [ ]:
df.groupby(by='SubGroup')[['AllConversions','ClickAssistedConversions',
                        'ImpressionAssistedConversions',
                        'CostEUR','Impressions','Clicks',
                        'Criteria']].aggregate({'AllConversions':sum,
                                                'ClickAssistedConversions':sum,
                                                'ImpressionAssistedConversions':sum,
                                                'CostEUR':sum,
                                                'Impressions':sum,
                                                'Clicks':sum,
                                                'Criteria':'count'})

In [ ]:
df[df['Group'] == 'Other'].groupby(by=(df[df['Group'] == 'Other']['ClickAssistedConversions'] +
                                       df[df['Group'] == 'Other']['ImpressionAssistedConversions']>0))[
  ['ClickAssistedConversions','ImpressionAssistedConversions','CostEUR',
   'Impressions','Clicks','Criteria']].aggregate({'ClickAssistedConversions':sum,
                                                  'ImpressionAssistedConversions':sum,
                                                  'CostEUR':sum,
                                                  'Impressions':sum,
                                                  'Clicks':sum,
                                                  'Criteria':'count'})

In [ ]:
def setGroup(x):
  if x['AllConversions'] > 0:
    return 'Converter'
  elif x['Clicks'] > 0:
    return 'Click'
  else:
    return 'Impressions'
  
def setSubGroup(x):
  if x['AllConversions'] > 99:
    return 'Converter - High'
  elif x['AllConversions'] > 9:
    return 'Converter - Mid'
  elif x['AllConversions'] > 2:
    return 'Converter - Low'
  elif x['AllConversions'] > 0:
    return 'Converter - Occasional'
  elif x['Clicks'] > 0:
    return 'Click'
  else:
    return 'Impressions'
  
df['Group'] = df.apply(setGroup,axis=1)
df['SubGroup'] = df.apply(setSubGroup,axis=1)

In [ ]:
df.hist('CPC',by='SubGroup',bins=100)

In [ ]:
with sns.axes_style({ 'axes.facecolor': 'white','axes.grid': False}):
  fig,ax = plt.subplots()
  data=df[df['Group'] == 'Click'].plot(x='CostEUR',y='CPC', 
                                     kind='hexbin',ax=ax,
                                     norm=colors.LogNorm(),
                                     cmap='PuBu_r',
                                     mincnt=1)

In [ ]:
metrics = ['CTR','CPC',u'QualityScore',u'AveragePosition',                          
           u'BounceRate',u'AveragePageviews',u'AverageTimeOnSite',
           u'PercentNewVisitors']

fig, ax_arr = plt.subplots( 4, 2,figsize=(10,10))

for i in range(len(metrics)):  
  sns.boxplot(data=df,y=metrics[i],x='Group',ax=ax_arr[i/2][(i%4)%2],showfliers=False)



In [ ]:
threshold = df[df['Group'] == 'Converter'][metrics].median()

In [ ]:
def setGroup(x):
  if x['AllConversions'] > 0:
    return 'Converter'
  elif x['Clicks'] > 0:
    return 'Click'
  else:
    return 'Impressions'
  
def setSubGroup(x,threshold=threshold):
  if x['AllConversions'] > 99:
    return 'Converter - High'
  elif x['AllConversions'] > 9:
    return 'Converter - Mid'
  elif x['AllConversions'] > 2:
    return 'Converter - Low'
  elif x['AllConversions'] > 0:    
    return 'Converter - Occasional'
  elif x['Clicks'] > 0:
    score = 0
    if x['CTR'] > threshold['CTR']:
      score+=1
      
    if x['CPC'] < threshold['CPC']:
      score+=1

    if x['QualityScore'] >= threshold['QualityScore']:
      score+=1
      
    if x['BounceRate'] < threshold['BounceRate']:
      score+=1    
    
    if x['AveragePageviews'] > threshold['AveragePageviews']:
      score+=1
      
    if x['AverageTimeOnSite'] > threshold['AverageTimeOnSite']:
      score+=1

    if x['PercentNewVisitors'] > threshold['PercentNewVisitors']:
      score+=1
      
    if x['AveragePosition'] < threshold['AveragePosition']:
      score+=1 
    
    return 'Click - {0:02d}'.format(score/3)  
  else:
    score = 0
    if x['QualityScore'] >= threshold['QualityScore']:
      score+=1
    
    if x['AveragePosition'] < threshold['AveragePosition']:
      score+=1

    return 'Impressions - {0:02d}'.format(score)  
  
df['Group'] = df.apply(setGroup,axis=1)
df['SubGroup'] = df.apply(setSubGroup,axis=1)

In [ ]:
df.groupby(by='SubGroup')[['AllConversions','ClickAssistedConversions',
                        'ImpressionAssistedConversions',
                        'CostEUR','Impressions','Clicks',
                        'Criteria']].aggregate({'AllConversions':sum,
                                                'ClickAssistedConversions':sum,
                                                'ImpressionAssistedConversions':sum,
                                                'CostEUR':sum,
                                                'Impressions':sum,
                                                'Clicks':sum,
                                                'Criteria':'count'})

In [ ]:
demo_context = gcp.Context.default()
ddm_context = gcp.Context(project_id='google.com:ddm-connectors-demo',
                          credentials=gcp.Context.default().credentials)

bigquery_dataset_name = 'aw_dataset'
bigquery_table_name = 'KEYWORDS_GROUPS'

# Define BigQuery dataset and table
dataset = bq.DataSet(name=bigquery_dataset_name,context=ddm_context)
table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name,context=ddm_context)

# Create BigQuery dataset
if not dataset.exists():
    dataset.create()

# Create or overwrite the existing table if it exists
table_schema = bq.Schema.from_dataframe(df[[u'AccountDescriptiveName', 
                                            u'CampaignName',u'AdGroupName', 
                                            u'Criteria', u'KeywordMatchType',
                                            u'Group',u'SubGroup']])
table.create(schema = table_schema, overwrite = True)

# Write the DataFrame to a BigQuery table
table.insert_data(df[[u'AccountDescriptiveName', u'CampaignName',u'AdGroupName', u'Criteria', u'KeywordMatchType',u'Group',u'SubGroup']])

In [ ]:
%bigquery sample --table google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_GROUPS --count 5

In [ ]:
%sql --module query_group
SELECT
  a.Date as Date,
  b.Group as Group,
  b.SubGroup as SubGroup,
  SUM(a.Impressions) as Impressions,
  SUM(a.Clicks) as Clicks,
  SUM(IF(a.AccountCurrencyCode = 'GBP', a.Cost*1.37, a.Cost*1.0))/1000000 as CostEUR,
  SUM(a.AllConversions) as AllConversions,
FROM
  [google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_PERFORMANCE_ANALYTICS]  a
JOIN
  [google.com:ddm-connectors-demo:aw_dataset.KEYWORDS_GROUPS]  b
ON
  a.AccountDescriptiveName = b.AccountDescriptiveName AND 
  a.CampaignName = b.CampaignName AND
  a.AdGroupName = b.AdGroupName AND
  a.Criteria = b.Criteria AND
  a.KeywordMatchType = b.KeywordMatchType
WHERE TRUE
AND NOT a.IsNegative
AND a.AdNetworkType2 = 'Google search'
AND a.Impressions > 0 
AND Month IN ('2015-08-01','2015-09-01','2015-10-01','2015-11-01')
AND a.AccountDescriptiveName IN ('Account Primario Volagratis','Volagratis Tratte Italia','Volagratis')
GROUP BY 1,2,3

In [ ]:
timeseries = bq.Query(query_group).to_dataframe()

In [ ]:
timeseries['Date'] = pd.to_datetime(timeseries['Date'])

In [ ]:
timeseries.set_index('Date',inplace=True)

In [ ]:
data = timeseries[['SubGroup','Impressions',
            'Clicks','AllConversions']].pivot(columns='SubGroup')

In [ ]:
data[[(u'Impressions',u'Impressions - 00'),(u'Impressions',u'Impressions - 01'),
      (u'Clicks',u'Click - 00'),(u'Clicks',u'Click - 01'),
      (u'Clicks',u'Click - 02'),(u'AllConversions',u'Converter - Occasional'),
      (u'AllConversions',u'Converter - Low'),
      (u'AllConversions',u'Converter - Mid'),
      (u'AllConversions',u'Converter - High')
      ]].plot()

In [ ]:
corr = data[[(u'Impressions',u'Impressions - 00'),(u'Impressions',u'Impressions - 01'),
      (u'Clicks',u'Click - 00'),(u'Clicks',u'Click - 01'),
      (u'Clicks',u'Click - 02'),(u'AllConversions',u'Converter - Occasional'),
      (u'AllConversions',u'Converter - Low'),
      (u'AllConversions',u'Converter - Mid'),
      (u'AllConversions',u'Converter - High')
      ]].corr()

In [ ]:
corr

In [ ]:
sns.heatmap(corr,)